Importacion de librerias.

Requests realiza la peticion al servidor.

BeautifulSoup analizar documentos HTML.

Pandas podemos representar datos tabulares con columnas con etiquetas y filas y series temporales.

In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

Se reliza un request de la url_inicial de la revista, url_root nos ayuda a genera un url completa a partir de la url raiz para esto se utiliza la libreria urljoin.

In [ ]:
url_inicial='http://www.revistarelaciones.com/index.php/relaciones/issue/archive'
url_root= 'http://www.revistarelaciones.com/index.php/relaciones/issue/archive'
r = requests.get(url_inicial)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

Se realiza la busqueda para obtener los urls de cada uno de los volumenes o archivos de la revista.

In [ ]:
box = soup.find('div', id='issues')
volumen=box.findAll('h4')
vol = [x.find('a').get('href')for x in volumen]
#vol=[urljoin(url_root,i) for i in vol]
vol

Se realiza un lista en la que la variable vol que contiene las urls de cada uno de 
los volumenes publicados de la revista, con estos urls obtenidos,  con la variable vol se implementa un ciclo en el cual se utiliza la variable vol2 de la cual se van acumulando las siguientes urls.

In [ ]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.findAll('div', id='issueCoverImage') 
    #volumen1=box1.findAll('h3', class_='title')
    vol1 = [x.find('a').get('href')for x in box1]
    #vol1=[urljoin(url_root,p) for p in vol1]
    vol2+=vol1

In [ ]:
vol2

Algunas revista nececita hacer mas busque da para llega a los articulos
ya que algunos maneja primero los volumens o achivos y de eso achivos se dirigen
a un urls que es la tabla contendios, donde se encuentran los articulos,por lo cual se comieza a realizar la misma busqueda que en vol2.

Se implementa otro ciclo con vol2, para esto se utiliza la variable de vol3 que recolecta las urls de los articulos, para comenzar con su scrapeo.

In [ ]:
vol3 =[]
for b in vol2: 
    url_inicial2=b 
    r2 = requests.get(url_inicial2)
    soup2 = BeautifulSoup(r2.text, 'html.parser')
    box2 = soup2.find('div', id='content') 
    volumen2=box2.findAll('tr', valign='top')
    vol2= [o.find('a').get('href')for o in volumen2]
    #links = [o.find('a')['href'] for o in box2.findAll('div',class_='tocTitle')]
    #vol1=[urljoin(url_root,p) for p in vol1]
    vol3+=vol2

In [ ]:
vol3

En esta funcion variable sopa se utiliza para tener las paginaciones de la revista y la variable url es guardan cada uno de url que ya se obtivieron de vol3.

In [ ]:
def get_url_items(sopa, url): 
    box2 = soup2.find('div', id='content') 
    volumen2=box2.findAll('tr', valign='top')
    vol2= [o.find('a').get('href')for o in volumen2]
    #vol=[urljoin(url_root,i) for i in vol]
    return vol3

Ahora se va acumulando cada uno de lo links e ir iterando en cada una
de las páginas, para traer cada unos de los links que se van a ir scrapeando.

In [ ]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

Obtiene el numero de los articulos que se enontraron en los primeros volumenes de la revista.

In [ ]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

Toma uno a uno de los links donde se encuantra la informacion de cada articulo para Scrapearlos.

In [ ]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

Mediante esta función resivisa cada uno de los links de los articulos en los cuales se encunntra la información
del articulo, por lo cual se aplican  cada uno de los metodos, que nos van a obtener lo que estamos requiriendo de dicho articulo.

In [ ]:
def scraper_book(url):
    content_book={}# diccionario que no va acumulando la imformacion que vamos poteniddo para el tada frame
    r=requests.get(url)
    tr='Relaciones Estudios de Historia y Sociedad'
    a='IV. Humanidades y Ciencias de la Conducta'
    te='Multidisciplinaria'
    s_item=BeautifulSoup(r.text,'html.parser')
    tituloR=tr
    if tituloR:
        content_book['Titulo Revista']=tituloR
    else :
        content_book['Titulo Revista']=None
    area=a
    if area:
        content_book['Area']=area
    else :
        content_book['Area']=None
    tematica=te
    if tematica:
        content_book['Tematica']=tematica
    else :
        content_book['Tematica']=None
    #titulo del libro
    try:
        titulo=s_item.find('h3').get_text(strip=True)
        content_book['Titulo Articulo']=titulo
    except AttributeError:
        content_book['Titulo Articulo']=None
    #obtencion de Resumen
    try:
        des=s_item.find('div', id='articleAbstract').get_text(strip=True)
        content_book['Resumen']=des.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #obtencion de Abstract
    try:
        de=s_item.find('div', class_='panel-boody').get_text(strip=True)
        content_book['Abstract']=de#.replace("Abstract","")
    except AttributeError:
        content_book['Abstract']=None
    #obtener link Articulos con la url, donde se guarda los urls donde se encutran la informacion de los articulos.
    try:
        urlA=url
        content_book['Link Articulo']=urlA
    except AttributeError:
        content_book['Link Articulo']=None
    #obtener link PDF
    try:
        ancla_link=s_item.find('div', id='articleFullText').find('a').get('href')
        content_book['Link PDF']=urljoin(url_root,ancla_link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

El list_scraper hace un scrapeo de la cada uno de los articulos, ya que en datos_book se acumulo la informacion obtenida de lo metodos anteriores.

In [ ]:
list_scraper=list_scraper[0:588]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

La variable datos_book es un listado en el que se recolecto la imformacion requerida por lo cual
se convierte en un DataFrame.

In [ ]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

En el caso de esta revista solo de deseaba obtener los articulos, por lo cual
en algunas ocaciones se obtienes cosas diferentes, para esto con la funcion isin lo que hace es 
eliminar mediante el Titulo Articulo los archivos que no se desean.

In [ ]:
df=df_catalogo[~ df_catalogo["Titulo Articulo"].isin(["Presentación","Editorial","Autores","Introducción","Corrigenda","Comentario","Revista completa","Preliminares","Portada e índice","Enlaces refback","Mañana o pasado","Pura imagen","Número completo", None])]
df

La imformacion solicitada se pasa a un CSV.

In [ ]:
df.to_csv('Revista154.csv', index=False)

Final del Código